In [1]:
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH # 导入段落对齐
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT # 导入段落对齐包
from docx import shared
from docx.shared import RGBColor #设置字体的颜色
from docx.shared import Cm, Inches,Pt
from docx.enum.table import WD_TABLE_ALIGNMENT  
from docx.enum.style import WD_STYLE
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.table import WD_ALIGN_VERTICAL   # 导入单元格垂直对齐
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls
import pandas as pd
from scipy import optimize
import os
import ns

from docx.table import _Cell
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import MultipleLocator

import sys,time

# 支持中文
plt.rcParams['font.sans-serif'] = ['SimSun']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

### 封装base类

In [2]:
class BaseWrod(object):
    """基础类"""
    doc = Document()   #创建一个新文档
    def __init__(self,name):
        self.filename = name
        self.background_color = "#FFFFFF"
        self.get_style()
        self.style_list = {}  # 声明一个字典，用户存放自定义的字体样式
        
    def footer(self):
        """添加页脚"""
#         self.doc.add_section()
#         self.doc.sections[2].footer_distance = Cm(1) # 设置页脚为1cm
#         footer2 = self.doc.sections[2].header
#         footer2.is_linked_to_previous = True
        
#     def create_element(self,name):
#         return OxmlElement(name)

#     def create_attribute(self,element, name, value):
#         element.set(ns.qn(name), value)


#     def add_page_number(self,run):
#         fldChar1 = self.create_element('w:fldChar')
#         self.create_attribute(fldChar1, 'w:fldCharType', 'begin')

#         instrText = self.create_element('w:instrText')
#         self.create_attribute(instrText, 'xml:space', 'preserve')
#         instrText.text = "PAGE"

#         fldChar2 = self.create_element('w:fldChar')
#         self.create_attribute(fldChar2, 'w:fldCharType', 'end')

#         run._r.append(fldChar1)
#         run._r.append(instrText)
#         run._r.append(fldChar2)

        

        
#     def header(self):
#         """添加页眉"""
#         self.doc.add_section()
#         self.doc.sections[2].header_distance = Cm(2) # 设置页眉为2cm
#         header2 = self.doc.sections[2].header  # 获取第2个节的页眉
#         header2.is_linked_to_previous = True
        
#         header_para = header2.paragraphs[-1]  # 获取页眉的第一个/最后一个段落
#         header_para.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 设置段落居中
#         run = header_para.add_run()  # 添加页眉内容
#         run.add_picture(
#             os.getcwd() + "/页眉图片.jpg",
#             height=docx.shared.Cm(1.4)
#         )
        
    def header2(self):
        header = self.doc.sections[0].header
        paragraph = header.paragraphs[0]
        run = paragraph.add_run()  # 添加页眉内容
        run.add_picture(
            os.getcwd() + "/页眉图片.jpg",
            height=Cm(1.4)
        )
        
        
    def get_style(self):
        """获取样式"""
        
        name_items = {
            "Normal":{"fontSize":"Pt(12)","font.color.rgb":"RGBColor(63,63,63)","paragraph_format.space_before":"Pt(5)","paragraph_format.space_after" :"Pt(5)","font.name":"u'宋体'"},
        }

    
    def update_style(self):
        """更新样式"""
        
        self.doc.styles["Normal"].font.name=u"宋体"  #设置全局字体
        self.doc.styles["Normal"]._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
        self.doc.styles["Normal"].font.color.rgb=RGBColor(63,63,63)  #设置正文全局颜色
        self.doc.styles["Normal"].font.size=Pt(12) #设置正文全局大小为12
        self.doc.styles['Normal'].paragraph_format.space_before =  Pt(5)  #段前距
        self.doc.styles['Normal'].paragraph_format.space_after =  Pt(5)   #段后距
        self.doc.styles["Heading 1"].font.size=Pt(28)  #设置全局1级标题的字体大小为28
        self.doc.styles["Heading 1"].font.color.rgb = RGBColor(11,156,158)  # 设置颜色
        self.doc.styles['Heading 1'].paragraph_format.space_before =  Pt(10) 
        self.doc.styles['Heading 1'].paragraph_format.space_after =  Pt(10)
        
        
    def add_style(self,name,para):
        """添加样式"""
        para["name"] = name
        style = self.doc.styles.add_style(**para)
        setattr(style,"fontName",'SimSun')
        self.style_list[name] = style
        
    def next_page(self):
        """翻到下一页"""
        self.doc.add_page_break()
        
                
    def save(self):
        """生成word"""
        self.doc.save(self.filename)
        
        
    def run(self):
        """
        生成报告文件
        """
        pass

class BaseTable(BaseWrod):
        # 初始化构造函数及单元格默认宽高并创建表格
    def __init__(self, data):
        self.data = data
        self.raw_num = len(data)
        self.col_num = len(data[0])
        self.createAndFillTable()
        self.alterCellHeight()
        self.formateCell()
        # 边框类型
        self.borderStyle = {
            'dash': {"sz": 10, "color": "#bebebe", "val": "dash"},
            'border': {"sz": 5, "color": "#bebebe", "val": "single"},
            'none': {"sz": 0, "color": "#ffffff", "val": "single"}
        }
        self.setWholeBorder()

    # 定义根据数据动态创建并填充表格的函数
    def createAndFillTable(self):
        self.table = self.doc.add_table(
            self.raw_num, self.col_num, style='Light Grid')
        self.table.alignment = WD_TABLE_ALIGNMENT.CENTER
        self.table.style.font.size = Pt(10)
        self.table.style.font.name = u'微软雅黑'
        self.table.style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.table.style.font.color.rgb = RGBColor(0x3f, 0x3f, 0x3f)
        for i, cell in enumerate(self.table._cells):
            cell.paragraphs[0].add_run(sum(self.data, [])[i])
        self.doc.add_paragraph()

        # 定义改变表格字体大小的函数
    def alterFontSize(self, size=6):
        for i in self.table._cells:
            i.paragraphs[0].runs[0].font.size = Pt(size)

    # 定义合并单元格的函数
    def mergeCells(self, raw_idx, start_idx, num):
        for i in range(start_idx, start_idx+num):
            cell_new = self.table.cell(raw_idx, start_idx).merge(
                self.table.cell(raw_idx, i))
        cell_new.text = self.table.cell(raw_idx, start_idx).text.rstrip()
        self.formateCell()

    # 定义单元格统一水平垂直居中的函数(直接用_cells)
    def formateCell(self):
        for i in self.table._cells:
            i.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
            i.paragraphs[0].alignment = WD_TABLE_ALIGNMENT.CENTER

    # 设置某列单元格左对齐(直接用_cells)
    def leftAlign(self, col_idx=1):
        for i in self.table.columns[col_idx].cells:
            i.paragraphs[0].alignment = WD_TABLE_ALIGNMENT.LEFT

    # 设置单元格底部对齐(直接用_cells)
    def bottomAlign(self):
        for i in self.table._cells:
            i.vertical_alignment = WD_ALIGN_VERTICAL.BOTTOM

    # 定义改变单元尺寸的函数

    def alterCellHeight(self, height=0.8):
        for i in range(self.raw_num):
            self.table.rows[i].height = Cm(height)

    def alterCellWidth(self, col_index=0, width=2):
        for cell in self.table.columns[col_index].cells:
            cell.width = Cm(width)

    # 设置背景色并改变字体颜色和加粗
    '''
    raw_idx:欲填充的单元格所在的行
    start(end)_idx:填充起始(终止)位置
    mode:包含连续(1)和离散(0)两种模式：separat continuous
    '''

    def fillBgColor(self, colorStr="#0b9c9e", raw_idx=0, start_idx=0, end_idx=-1, mode=1):
        shading_list = locals()
        end_idx = self.col_num if end_idx == -1 else end_idx
        fill_list = range(start_idx, self.col_num) if mode else range(
            start_idx, self.col_num, 2)
        for i in list(fill_list):
            self.table.rows[raw_idx].cells[i].paragraphs[0].runs[0].font.bold = True
            self.table.rows[raw_idx].cells[i].paragraphs[0].runs[0].font.name = u'微软雅黑'
            self.table.rows[raw_idx].cells[i].paragraphs[0].runs[0].font.color.rgb = RGBColor(
                0xff, 0xff, 0xff) if colorStr == "#0b9c9e" else RGBColor(0x3f, 0x3f, 0x3f)
            shading_list['shading_elm_'+str(i)] = parse_xml(
                r'<w:shd {} w:fill="{bgColor}"/>'.format(nsdecls('w'), bgColor=colorStr))
            self.table.rows[raw_idx].cells[i]._tc.get_or_add_tcPr().append(
                shading_list['shading_elm_'+str(i)])

    def set_cell_border(self, cell, **kwargs):
        """
        Set cell`s border
        Usage:

        set_cell_border(
            cell,
            top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
            bottom={"sz": 12, "color": "#00FF00", "val": "single"},
            start={"sz": 24, "val": "dashed", "shadow": "true"},
            end={"sz": 12, "val": "dashed"},
        )
        """

        tc = cell._tc
        tcPr = tc.get_or_add_tcPr()

        # check for tag existnace, if none found, then create one
        tcBorders = tcPr.first_child_found_in("w:tcBorders")
        if tcBorders is None:
            tcBorders = OxmlElement('w:tcBorders')
            tcPr.append(tcBorders)

        # list over all available tags
        for edge in ('start', 'top', 'end', 'bottom', 'left', 'right', 'insideH', 'insideV'):
            edge_data = kwargs.get(edge)
            if edge_data:
                tag = 'w:{}'.format(edge)

                # check for tag existnace, if none found, then create one
                element = tcBorders.find(qn(tag))
                if element is None:
                    element = OxmlElement(tag)
                    tcBorders.append(element)

                # looks like order of attributes is important
                for key in ["sz", "val", "color", "space", "shadow"]:
                    if key in edge_data:
                        element.set(qn('w:{}'.format(key)),
                                    str(edge_data[key]))

    # 设置整体的边框风格的函数(直接用_cells)
    def setWholeBorder(self):
        for i in self.table._cells:
            self.set_cell_border(
                i, top=self.borderStyle.get('border'), bottom=self.borderStyle.get('border'), left=self.borderStyle.get('border'), right=self.borderStyle.get('border'))

    # 设置顶部粗框线
    def setOutlineTop(self, sz=28):
        for i in self.table.rows[0].cells:
            self.set_cell_border(i, top={
                "sz": sz, "color": "#0b9c9e", "val": "single"})

    # 去除垂直边框(直接用_cells)
    def clearVborder(self):
        for i in self.table._cells:
            self.set_cell_border(i, left=self.borderStyle.get(
                'none'), right=self.borderStyle.get('none'))

    # 去除水平边框(直接用_cells)
    def clearHborder(self):
        for i in self.table._cells:
            self.set_cell_border(i, top=self.borderStyle.get(
                'none'), bottom=self.borderStyle.get('none'))

    # 添加底部水平边框
    def addBottomBorder(self, col_idx=1):
        for i in self.table.columns[col_idx].cells:
            self.set_cell_border(i, bottom=self.borderStyle.get('border'))

    # 设置底部粗框线

    def setOutlineBottom(self, sz=14):
        for i in self.table.rows[-1].cells:
            self.set_cell_border(i, bottom={
                "sz": sz, "color": "#0b9c9e", "val": "single"})


# 封装相同长表类
class LongTable(BaseTable):
    def __init__(self, data):
        super().__init__(data)
        self.mergeCells(0, 4, 3)
        self.mergeCells(0, 0, 4)
        self.fillBgColor()
        self.fillBgColor(raw_idx=1, end_idx=3, colorStr="#daeef3")
        self.fillBgColor(raw_idx=1, start_idx=4, colorStr="#f1f1f1")


# 封装相同短表类
class ShortTable(BaseTable):
    def __init__(self, data):
        super().__init__(data)
        self.mergeCells(0, 3, 3)
        self.mergeCells(0, 0, 3)
        self.fillBgColor()
        self.fillBgColor(raw_idx=1, end_idx=2, colorStr="#daeef3")
        self.fillBgColor(raw_idx=1, start_idx=3, colorStr="#f1f1f1")


### 定义业务类

In [3]:
class EtiologyReport(BaseWrod):
    """病原报告"""
    def __init__(self,name):
        super().__init__(name)
        
                        
    def home_page(self):
        """首页"""
        
        self.doc.add_picture(r"./首页/top.jpg",width = Cm(15.5),height = Cm(6.75))
        self.doc.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  #设置图片居中
        
        #设置标题样式
        title1_style = self.doc.styles.add_style('title1_style', WD_STYLE_TYPE.PARAGRAPH)
        title1_style.paragraph_format.space_before =  Pt(15) 
        title1_style.paragraph_format.space_after =  Pt(15)
        title1_style.font.name=u"微软雅黑" 
        title1_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        title1_style.font.size=Pt(29)
        title1_style.font.bold = True
        title1_style.font.color.rgb = RGBColor(11,156,158)
        title1_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        self.doc.add_paragraph('病原微生物宏基因组（RNA）',title1_style)
        
        
        
        title2_style = self.doc.styles.add_style('title2_style', WD_STYLE_TYPE.PARAGRAPH)
        title2_style.paragraph_format.space_before =  Pt(10) 
        title2_style.paragraph_format.space_after =  Pt(60)
        title2_style.font.name=u"微软雅黑" 
        title2_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        title2_style.font.size=Pt(20)
        title2_style.font.color.rgb = RGBColor(63,63,63)
        title2_style.font.bold = True
        title2_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        self.doc.add_paragraph('— 检测报告 —',title2_style)
        
        
        
        first_page = [
            ["受 检 者：", ""],
            ["送检单位：",""],
            ["检测单位：","广州达安临床检验中心"],
            ["接收日期：",""],
            ["报告日期：",""],
        ]
        
        table0 = BaseTable(first_page)
        table0.clearHborder()
        table0.clearVborder()
        table0.addBottomBorder()
        table0.alterFontSize(12)
        table0.alterCellHeight(1.2)
        table0.alterCellWidth(0,3)
        table0.alterCellWidth(1, 8)
        table0.bottomAlign()
        

        
        
    def declaration_page(self):
        """声明页"""
        
        self.title3_style = self.doc.styles.add_style('title3_style', WD_STYLE_TYPE.PARAGRAPH)
        self.title3_style.paragraph_format.space_before =  Pt(20) 
        self.title3_style.paragraph_format.space_after =  Pt(20)
        self.title3_style.paragraph_format.line_spacing = Pt(20)  #行间距
        self.title3_style.font.size=Pt(18)
        self.title3_style.font.name=u"微软雅黑" 
        self.title3_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.title3_style.font.bold = True
        self.title3_style.font.color.rgb = RGBColor(11,156,158)
        self.title3_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        self.doc.add_paragraph('致客户',self.title3_style)
    
        
        self.content1_style = self.doc.styles.add_style('content1_style', WD_STYLE_TYPE.PARAGRAPH)
        self.content1_style.paragraph_format.space_before =  Pt(20) 
        self.content1_style.paragraph_format.space_after =  Pt(20)
        self.content1_style.font.name=u"微软雅黑" 
        self.content1_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.content1_style.font.size=Pt(14)
        self.content1_style.font.bold = True
        self.content1_style.font.color.rgb = RGBColor(0,0,0)
        self.content1_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT
        self.doc.add_paragraph("尊敬的客户：",self.content1_style)
        
        
        self.content2_style = self.doc.styles.add_style('content2_style', WD_STYLE_TYPE.PARAGRAPH)
        self.content2_style.paragraph_format.space_before =  Pt(10)  #段前距
        self.content2_style.paragraph_format.space_after =  Pt(10)  #段后距
        self.content2_style.paragraph_format.line_spacing = Pt(25)  #行间距
        self.content2_style.font.size=Pt(12)
        self.content2_style.font.name=u"微软雅黑" 
        self.content2_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.content2_style.paragraph_format.first_line_indent = self.content2_style.font.size *2 #首行缩进
        self.content2_style.font.color.rgb = RGBColor(63,63,63)
        self.content2_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        
        contents = [
            "您好！",
            "感谢您对我们的信任。",
            "病原微生物宏基因组检测是一种不依赖传统微生物培养而广泛分析临床样本中微生物（细菌、真菌、 病毒、寄生虫等）的高通量测序检测技术。通过临床样本的中微生物总体基因组（DNA或RNA）的提取、高通量测序和数据分析，可获得疑似致病微生物的种属信息。 ",
            "达安临床检验中心本着严谨科学的态度，结合目前最新的指南、共识和临床研究现状，采用高通量测序技术和智能化算法分析，对您的样本进行病原微生物宏基因组检测，可辅助医生进行临床病原微生物诊断，致力于临床精准诊断和精准治疗。",
            "本检测报告是临床医生在临床诊断和制定治疗方案时强有力的参考依据。除此之外，还需结合患者的其他情况进行综合考虑，最终的诊断结果和治疗方案需要由主治医生进行综合判断和选择。",
            "希望您能将您的宝贵意见或建议及时反馈给我们。我们的免费服务电话是：800-830-3620（周一至周五，8:30--17:30），我们将竭诚为您服务！"
        ]
        for content in contents:
            self.doc.add_paragraph(content, self.content2_style)
        
        
        self.content3_style = self.doc.styles.add_style('content3_style', WD_STYLE_TYPE.PARAGRAPH)
        self.content3_style.paragraph_format.space_before =  Pt(30) 
        self.content3_style.paragraph_format.space_after =  Pt(20)
        self.content3_style.font.name=u"微软雅黑" 
        self.content3_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.content3_style.font.size=Pt(14)
        self.content3_style.font.bold = True
        self.content3_style.font.color.rgb = RGBColor(0,0,0)
        self.content3_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        self.doc.add_paragraph("广州达安临床检验中心",self.content3_style)
        
        
    def base_info(self):
        """受检者信息"""
        self.title4_style = self.doc.styles.add_style('title4_style', WD_STYLE_TYPE.PARAGRAPH)
        self.title4_style.paragraph_format.space_before =  Pt(20) 
        self.title4_style.paragraph_format.space_after =  Pt(10)
        self.title4_style.font.name=u"微软雅黑" 
        self.title4_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.title4_style.font.size=Pt(14)
        self.title4_style.font.bold = True
        self.title4_style.font.color.rgb = RGBColor(11,156,158)
        self.title4_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT

        baseinfo_title1 = "一、受检者信息"
        self.doc.add_paragraph(baseinfo_title1, self.title4_style)

        # 添加受检者信息表
        patient_info = [["姓名", "", "性别", "男", "年龄", "69岁"], [
    '送检医院', '梅州市人民医院', '', '', '', ''], ['送检科室', '呼吸与危重症医学 (重症监护科)', '送检医生', '-', '门诊号/住院号', '-'], ['样本类型', '肺泡灌洗液', '', '', '条码号', ''], ['临床诊断', '/', '', '', '', ''], ['个人病史', '/', '', '', '', ''], ['其他检测结果', '/', '', '', '', ''], ["采样日期", "", "接受日期", "", "报告日期", ""]]
        table1 = BaseTable(patient_info)
        table1.fillBgColor('#f1f1f1', mode=0)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=1)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=2)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=3)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=4)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=5)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=6)
        table1.fillBgColor('#f1f1f1', mode=0, raw_idx=7)
        table1.mergeCells(1, 1, 5)
        table1.mergeCells(3, 1, 3)
        table1.mergeCells(4, 1, 5)
        table1.mergeCells(5, 1, 5)
        table1.mergeCells(6, 1, 5)
        table1.setOutlineTop()
        table1.setOutlineBottom()

        
            
        baseinfo_title2 = "二、检测结果摘要"
        self.doc.add_paragraph(baseinfo_title2, self.title4_style)
        
        
        # 根据病原信息创建病原检出表
        etiology_info = [['各类疑似致病的病原微生物检出结果', '', '', ''], ['细菌', '病毒', '真菌', '寄生虫'], [
            "铜绿假单胞菌(95549)\n金黄色葡萄球菌 (1661)\n蜡样芽孢杆菌复合群 (1189)\n大肠埃希菌 (158)\n肺炎克雷伯菌 (41)\n常见微生态菌群\n其他病原请见详情列表", "人类疱疹病毒 1 型(HSV1) (9)\n人类疱疹病毒 5 型(CMV)(6)", "未检出疑似病原体\n常见微生态菌群\n其他病原请见详情列表", '未检出']]
        table2 = BaseTable(etiology_info)
        table2.mergeCells(0, 0, 4)
        table2.fillBgColor()
        table2.fillBgColor(raw_idx=1, mode=0, colorStr='#daeef3')
        table2.fillBgColor(raw_idx=1, mode=0, colorStr='#f1f1f1', start_idx=1)
        table2.clearHborder()
        table2.setOutlineTop(5)
        table2.setOutlineBottom()


        
        self.notice_style = self.doc.styles.add_style('notice_style', WD_STYLE_TYPE.PARAGRAPH)
        self.notice_style.paragraph_format.space_before =  Pt(5)  #段前距
        self.notice_style.paragraph_format.space_after =  Pt(5)  #段后距
        self.notice_style.paragraph_format.line_spacing = Pt(15)  #行间距
        self.notice_style.font.name=u"微软雅黑" 
        self.notice_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.notice_style.font.size=Pt(10)
        self.notice_style.font.color.rgb = RGBColor(63,63,63)
        self.notice_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        
        notice_content = [
            "*注：",
            "1. 括号内为检出的特异性序列(reads)数；详细检测结果见 3.1-3.6。 ",
            "2. 对于免疫低下/缺陷患者，请注意关注微生态菌群(3.6)；部分重要的条件致病微生物(▲)见微生态菌群列表。",
            "3. 建议临床结合患者症状和其它辅助检测，进一步确认患者的感染情况。"
        ]
        
        for content in notice_content:
            self.doc.add_paragraph(content, self.notice_style)
        
        table_data = [
            ["检测者：黎志浩", "审核者："],
        ]
        
        # table2 = self.createTable(table_data)
        table2 = BaseTable(table_data)
        
        last_content =  '本报告仅供临床科研参考，本次检测仅对本样本负责，如有疑义请在七个工作日内与我们联系。'
        self.doc.add_paragraph(last_content , self.notice_style)
        
        
    def test_result_detail(self):
        """检测结果详述"""
        baseinfo_title3 = '三、检测结果详述'
        self.doc.add_paragraph(baseinfo_title3, self.title4_style)
        
        
        self.title5_style = self.doc.styles.add_style('title5_style', WD_STYLE_TYPE.PARAGRAPH)
        self.title5_style.paragraph_format.space_before =  Pt(10) 
        self.title5_style.paragraph_format.space_after =  Pt(10)
        self.title5_style.font.name=u"微软雅黑" 
        self.title5_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.title5_style.font.size=Pt(12)
        self.title5_style.font.bold = True
        self.title5_style.font.color.rgb = RGBColor(63,63,63)
        self.title5_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        baseinfo_title31 = '3.1 细菌检测结果'
        self.doc.add_paragraph(baseinfo_title31, self.title5_style)
        
        # 根据细菌检测结果
        check_result = [['属', '', '', '', '种', '', ''], [
            "类型", "属名", "相对丰度%", "reads数", "种名", "鉴定置信度", "reads数"
        ], [
            "G-", "假单胞菌属\nPseudomonas", "88.70", "166040", "铜绿假单胞菌\nPseudomonas aeruginosa", "0.99", "95549"
        ], [
            "G-", "假单胞菌属\nPseudomonas", "88.70", "166040", "铜绿假单胞菌\nPseudomonas aeruginosa", "0.99", "95549"
        ], [
            "G-", "假单胞菌属\nPseudomonas", "88.70", "166040", "铜绿假单胞菌\nPseudomonas aeruginosa", "0.99", "95549"
        ], [
            "G-", "假单胞菌属\nPseudomonas", "88.70", "166040", "铜绿假单胞菌\nPseudomonas aeruginosa", "0.99", "95549"
        ], [
            "G-", "假单胞菌属\nPseudomonas", "88.70", "166040", "铜绿假单胞菌\nPseudomonas aeruginosa", "0.99", "95549"
        ]]

        table3 = LongTable(check_result)
        
        notice_content = [
            "注：",
           " 1. G+为革兰氏染色阳性菌，G-为革兰氏染色阴性菌（下同）。",
            "2. 相对丰度：该病原体相应分类中基因组相对比例（下同）。" ,
            "3. reads 数：高通量测序序列唯一比对到某属或种微生物特异性序列数（下同）。",
            "4. 鉴定置信度：根据序列比对，在样本中鉴定该病原体的可信度（下同）。"
        ]
        
        for content in notice_content:
            self.doc.add_paragraph(content, self.notice_style)
        
        
        # 3.2真菌检测结果
        baseinfo_title32 = '3.2 真菌检测结果'
        self.doc.add_paragraph(baseinfo_title32, self.title5_style)
        
        # 真菌检测结果
        check_result2 = [['属', '', '', '种', '', ''], ["属名", "相对丰度%", "reads数", "种名", "鉴定置信度", "reads数"], [
            "-", "-", "-", "-", "-", "-"
        ]]

        table4 = ShortTable(check_result2)
        
        # 3.3 病毒检测结果
        baseinfo_title33 = '3.3 病毒检测结果'
        self.doc.add_paragraph(baseinfo_title33, self.title5_style)
        
        # 病毒检测结果
        check_result3 = [['名称', '置信度(%)', 'reads数'], ['人类疱疹病毒 1 型(HSV1)\nHuman alphaherpesvirus 1', '0.99', '9'], [
            '人类疱疹病毒 5 型(CMV)\nHuman betaherpesvirus 5', '0.99', '6']]
        table5 = BaseTable(check_result3)
        table5.fillBgColor()
        
        
        # 3.4 寄生虫检测结果
        baseinfo_title34 = '3.4 寄生虫检测结果'
        self.doc.add_paragraph(baseinfo_title34, self.title5_style)
        
        # 寄生虫检测结果
        table6 = ShortTable(check_result2)
        
        #3.5 疑似人体微生态菌群检测结果
        baseinfo_title35 = '3.5 疑似人体微生态菌群检测结果'
        self.doc.add_paragraph(baseinfo_title35, self.title5_style)
        
        table7 = LongTable(check_result)
        
        notice_content = [
            "注：人体微生态菌群为存在于人体皮肤、呼吸道、口腔、胃肠道、泌尿道的微生物，多为条件致病菌，正常条件下与人体共生，在免疫力低下/缺陷的患者中具有潜在致病性。"
        ]
        
        for content in notice_content:
            self.doc.add_paragraph(content, self.notice_style)
        
        
        #3.6 疑似耐药基因检测结果
        baseinfo_title36 = '3.6 疑似耐药基因检测结果'
        self.doc.add_paragraph(baseinfo_title36, self.title5_style)
        
        # 疑似耐药基因检测结果
        check_result4 = [["基因名称", "疑似对应物种", "reads数", "耐药类型", "相关药物"], [
            "-", "-", "-", "-", "-"
        ]]
        table8 = BaseTable(check_result4)
        table8.fillBgColor()

        notice_content = [
            "注：耐药基因与表型之间可能存在差异，检出耐药基因并不能明确该菌一定对相应药物耐药，检测结果仅供参考，请以临床医生指导用药为准。更多耐药基因信息详见：https://cge.cbs.dtu.dk/services/ResFinder/；https://card.mcmaster.ca/"
        ]
        
        for content in notice_content:
            self.doc.add_paragraph(content, self.notice_style)
            
    
    def test_by_declare(self):
        """检出病原体说明"""
        baseinfo_title4 = '四、检出病原体说明'
        self.doc.add_paragraph(baseinfo_title4, self.title4_style)
        
        
        
        contents = [
            "铜绿假单胞菌 (Pseudomonas aeruginosa)：革兰阴性杆菌，假单胞菌属。铜绿假单胞菌广泛分布于水、空气、土壤，以及正常人体皮肤、呼吸道与肠道粘膜中，为条件致病菌。因手术、化疗、放疗、激素治疗等原因人体抵抗力低下时容易引起感染。可引起烧伤创面感染、肺部感染、泌尿道感染、中耳炎、脑膜炎、菌血症等。是院内感染的常见病原体。",
            
        ]
        for content in contents:
            self.doc.add_paragraph(content, self.content2_style)
        
        
        #插入图片
        self.doc.add_picture(r"4-1.jpg",width = Cm(10),height = Cm(7)) 
        self.doc.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  #设定图片居中
        
        notice_content = [
            "注：以上所展示的覆盖度图为针对特定微生物绘制图谱，反映比对到该微生物的序列在其基因组上的分布情况，横坐标代表该微生物的基因组大小，纵坐标代表不同基因组区段内检出的序列数。"
        ]
        for content in notice_content:
            self.doc.add_paragraph(content, self.notice_style)
        
    def the_appendix(self):
        """附录"""
        
        appendix_info1 = '附录'
        self.doc.add_paragraph(appendix_info1,self.title3_style)
        
        #检测质量控制
        appendix_info_title1 = '检测质量控制'
        
        self.appendix_title1_style = self.doc.styles.add_style('appendix_title1_style', WD_STYLE_TYPE.PARAGRAPH)
        self.appendix_title1_style.paragraph_format.space_before =  Pt(20) 
        self.appendix_title1_style.paragraph_format.space_after =  Pt(20)
        self.appendix_title1_style.paragraph_format.line_spacing = Pt(20)  #行间距
        self.appendix_title1_style.font.name=u"微软雅黑" 
        self.appendix_title1_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.appendix_title1_style.font.size=Pt(14)
        self.appendix_title1_style.font.bold = True
        self.appendix_title1_style.font.color.rgb = RGBColor(63,63,63)
        self.appendix_title1_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT
        
        self.doc.add_paragraph(appendix_info_title1, self.appendix_title1_style)
        
        
        #1. 全流程质控
        appendix_info_title11 = '1. 全流程质控'
        self.doc.add_paragraph(appendix_info_title11, self.title5_style)
        
        # 全流程质控
        whole_qc = [['质控参数', '质控结果'], ['内参', '合格'],
                    ['质控品', '合格'], ['防错标签(UMSI)', '合格']]
        table9 = BaseTable(whole_qc)
        table9.fillBgColor()
        table9.setOutlineBottom()

        appendix_info_title21 = '2. 测序数据质控'
        self.doc.add_paragraph(appendix_info_title21, self.title5_style)
        
        # 测序数据质控
        seq_qc = [['总序列(reads)数', '总碱基数', 'Q30 比例(%)', '测序质量评估'],
                ['60924229', '1086504400', '92.66', '合格']]
        table10 = BaseTable(seq_qc)
        table10.fillBgColor()
        table10.setOutlineBottom()

        notice_content = [
            "注：",
            "1. 总序列（reads）数：样本经过高通量测序并过滤低质量数据后，获得的所有测序序列的数目。",
            "2. 总碱基数：样本经过高通量测序并过滤低质量数据后，获得的所有测序序列的总碱基的数目。",
            "3. 碱基质量值（Q）是衡量测序质量的重要指标，Q越高代表碱基被测错的概率越小，其中Q30为测序错误的概率在0.1%的 质量值。Q30比例为样本测序数据中质量值≥Q30的碱基比例。"
        ]
        for notice in notice_content:
            self.doc.add_paragraph(notice, self.notice_style)
            
            
            
            
        #检测分析性能
        appendix_info_title2 = '检测分析性能'
        self.doc.add_paragraph(appendix_info_title2, self.appendix_title1_style)
        # 检测分析性能
        analyze = [['检测限', '灵敏度', '特异性'], [
            "50-500 copies/mL", "50 copies/mL", "99%"
        ]]
        table11 = BaseTable(analyze)
        table11.fillBgColor()
        table11.setOutlineBottom()
                
        #检测方法简介
        appendix_info_title3 = '检测方法简介'
        self.doc.add_paragraph(appendix_info_title3, self.appendix_title1_style)
        
        contents = [
            "病原宏基因组学是一种新的不依赖于培养的广泛分析临床样本中微生物组的高通量测序方法[1]。适用于不明原因发热、疑难危重以及免疫缺陷感染患者[2-3]，目前已有多篇用于脓毒症、脑膜炎、呼吸系统感染等方面的报道[4-6]。本检测基于高通量测序平台，数据库涵盖19036种微生物（9855种细菌、6926种病毒、1582种真菌、312种寄生虫、分枝杆菌复合群中的177种常见致病菌和184种支原体/衣原体），通过提取临床样本中的核酸序列，构建测序文库，对样本中核酸序列进行测序，再通过微生物专用数据库进行比对分析[9经过智能化算法获得疑似致病微生物的种属信息，并提供全面深入的报告参数，辅助临床诊疗决策[7]。",
        ]
        for content in contents:
            self.doc.add_paragraph(content, self.content2_style)
            
        
    def the_references(self):
        """主要参考文献"""
        appendix_info2 = '主要参考文献'
        self.doc.add_paragraph(appendix_info2,self.title3_style)

        # 文献列表
        article_data = [
            ["1" ,"Wilson, M.R., et al., Actionable diagnosis of neuroleptospirosis by next-generation sequencing. N Engl J Med, 2014.370(25): p.2408-17."],
            ["2" , "Dekker, J.P., Metagenomics for Clinical Infectious Disease Diagnostics Steps Closer to Reality. J Clin Microbiol, 2018.56(9)."],
            ["3" ,"Thoendel, M.J., et al., Identification of Prosthetic Joint Infection Pathogens Using a Shotgun Metagenomics Approach. Clin Infect Dis, 2018.67(9): p.1333-1338."],
            ["4","Brown, J.R., T. Bharucha, and J. Breuer, Encephalitis diagnosis using metagenomics: application of next generation sequencing for undiagnosed cases. J Infect, 2018.76(3): p.225-240."],
            ["5","Parize, P., et al., Untargeted next-generation sequencing-based first-line diagnosis of infection in immunocompromised adults: a multicentre, blinded, prospective study. Clin Microbiol Infect, 2017.23(8): p.574 e1-574e6."],
            ["6","Blauwkamp, T. A., et al., Analytical and clinical validation of a microbial cell-free DNA sequencing test for infectious disease. Nat Microbiol, 2019,4(4), p.663."],
            ["7","Miller, S., et al. Laboratory validation of a clinical metagenomic sequencing assay for pathogen detection in cerebrospinal fluid. Genome Res,2019."],
            ["8","Langelier ,C., et al. Integrating host response and unbiased microbe detection for lower respiratory tract infection diagnosis in critically ill adults. P Natl Acad of Sci USA, 2018,115(52):E12353-E12362."],
            ["9","Gu, W., S. Miller, and C.Y. Chiu, Clinical Metagenomic Next-Generation Sequencing for Pathogen Detection. Annu Rev Pathol, 2019. 14: p.319-338."],
            ["10","Miao, Q., et al., Microbiological Diagnostic Performance of Metagenomic Next-generation Sequencing When Applied to Clinical Practice. Clin Infect Dis, 2018. 67(suppl_2): p.S231-S240."],
            ["11","Xie, Y., et al., Next generation sequencing for diagnosis of severe pneumonia: China,2010-2018. J Infect, 2019. 78(2): p.158-169."]
        ]
        
        table12 = BaseTable(article_data)
        table12.alterFontSize(8)
        table12.alterCellWidth(0, 0.6)
        table12.alterCellWidth(1, 14.4)
        table12.alterCellHeight(1.5)
        table12.setOutlineBottom()
        table12.setOutlineTop()
        table12.clearVborder()
        table12.leftAlign()
    
    def the_notice(self):
        """检测报告使用须知"""
        appendix_info3 = '检测报告使用须知'
        self.doc.add_paragraph(appendix_info3,self.title3_style)
        
        self.content4_style = self.doc.styles.add_style('content4_style', WD_STYLE_TYPE.PARAGRAPH)
        self.content4_style.paragraph_format.space_before =  Pt(20)  #段前距
        self.content4_style.paragraph_format.space_after =  Pt(20)  #段后距
        self.content4_style.paragraph_format.line_spacing = Pt(25)  #行间距
        self.content4_style.font.size=Pt(12)
        self.content4_style.font.name=u"微软雅黑" 
        self.content4_style._element.rPr.rFonts.set(qn('w:eastAsia'), u'微软雅黑')
        self.content4_style.font.color.rgb = RGBColor(63,63,63)
        self.content4_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        
        contents = [
            "1. 本检测报告仅对本次送检样本负责。",
            "2. 样本采集，保存，运输等环节的不当操作可能影响本产品检测性能。",
            "3. 本检测结果仅对该次送检样本负责。方法学的技术局限性、微生物载量、当前数据库构建局限或临床用药等因素均可能使得检测结果未涵盖样本中所有的微生物。",
            "4. 本检测非临床常规检测项目，仅供临床参考，临床相关解释须咨询临床医生。任何药物或治疗方案的 制定均需由主治医生结合患者的其他情况，综合考虑。",
            "5. 如对检测结果有疑问，请于收到报告后 7 个工作日内与我们联系。",
        
        ]
        for content in contents:
            self.doc.add_paragraph(content, self.content4_style)
            
        
        self.doc.add_picture(r"尾页.jpg",width = Cm(15),height = Cm(20)) 
        self.doc.paragraphs[-1].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  #设定图片居中
        
        
        
    def run(self):
        
        self.home_page()
        self.next_page()
        self.declaration_page()
        self.next_page()
        self.base_info()
        self.header2()
        self.add_page_number(self.doc.sections[0].footer.paragraphs[0].add_run())
        self.next_page()
        self.test_result_detail()
        self.next_page()
        self.test_by_declare()
        self.next_page()
        self.the_appendix()
        self.next_page()
        self.the_references()
        self.next_page()
        self.the_notice()
        
        self.save()
        
        
        
        

In [4]:
rid = int(time.time())
filename = str(rid) + ".doc"
elp = EtiologyReport(filename)
elp.run()

KeyError: "no style with name 'Light Grid'"